In [1]:
import os
import sys
import pymssql
import locale

sys.path.append(os.path.abspath(".."))

import pandas as pd
import numpy as np


from dotenv import load_dotenv
from num2words import num2words
from docxtpl import DocxTemplate
from docx2pdf import convert
from utils_ccd import get_connection, generate_pdf_office

load_dotenv()
conn = get_connection()
locale.setlocale(locale.LC_ALL, 'pt_BR.UTF-8')

c:\Users\05911205424\Documents\Dev\ccd\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


'pt_BR.UTF-8'

In [2]:
# mês atual, ano atual
from datetime import datetime
now = datetime.now()
mes_atual = now.month
ano_atual = now.year


#somar valores das multas

In [3]:
descontos_sql = f''' 
SELECT 
    pro.numero_processo,
    pro.ano_processo,
    pro.assunto,
    pm.Descricao as descricao,
    CONCAT(gp.Nome, ' (CPF: ', gp.Documento, ')') as nome,
    processo.dbo.fn_Exe_RetornaValorAtualizado(ed.IdDebito) as valor_multa,
    orgao_info.nome_orgao,
    ed.CodigoStatusDivida as status_divida
FROM processo.dbo.Processos pro
INNER JOIN processo.dbo.Pro_MarcadorProcesso pmp 
    ON pmp.IdProcesso = pro.IdProcesso
INNER JOIN processo.dbo.Pro_Marcador pm 
    ON pmp.IdMarcador = pm.IdMarcador
LEFT JOIN processo.dbo.Exe_Debito ed 
    ON ed.IdProcessoExecucao = pro.IdProcesso
LEFT JOIN processo.dbo.Exe_DebitoPessoa edp 
    ON edp.IDDebito = ed.IdDebito
LEFT JOIN processo.dbo.GenPessoa gp 
    ON gp.IdPessoa = edp.IDPessoa
OUTER APPLY (
    SELECT TOP 1 vspfr.nome_orgao
    FROM BdDIP.dbo.vwSiaiPessoalFolhaResumida vspfr
    WHERE vspfr.cpf = gp.Documento COLLATE SQL_Latin1_General_CP1_CI_AS
    AND vspfr.mes = {mes_atual - 1} AND vspfr.ano = {ano_atual}
    ORDER BY vspfr.ano DESC, vspfr.mes DESC
) AS orgao_info
WHERE 
    pro.setor_atual = 'CCD'
    AND pm.Descricao LIKE 'DESCONTO EM FOLHA - Implementar'
    AND ed.CodigoTipoDebito NOT IN (1, 3)
'''
processos_descontos = pd.read_sql(descontos_sql, conn)

In [4]:
processos_descontos

,numero_processo,ano_processo,assunto,descricao,nome,valor_multa,nome_orgao,status_divida
0,003037,2025,EXECUÇÃO DA DECISÃO PROFERIDA NOS AUTOS DO PRO...,DESCONTO EM FOLHA - Implementar,MARCOS ANTÔNIO CABRAL (CPF: 32403143472),1195.2656,ASSEMBLEIA LEGISLATIVA DO ESTADO DO RN,1
1,002819,2025,EXECUÇÃO DA DECISÃO PROFERIDA NOS AUTOS DO PRO...,DESCONTO EM FOLHA - Implementar,CARLOS ALBERTO JÁCOME DE AQUINO (CPF: 46554408...,11308.9867,SECRETARIA DE ESTADO DA TRIBUTAÇÃO,1
2,002819,2025,EXECUÇÃO DA DECISÃO PROFERIDA NOS AUTOS DO PRO...,DESCONTO EM FOLHA - Implementar,CARLOS ALBERTO JÁCOME DE AQUINO (CPF: 46554408...,5315.8773,SECRETARIA DE ESTADO DA TRIBUTAÇÃO,1
3,002818,2025,EXECUÇÃO DA DECISÃO PROFERIDA NOS AUTOS DO PRO...,DESCONTO EM FOLHA - Implementar,FRANCISCO BRUNO FERREIRA DA COSTA (CPF: 077605...,828.5144,ASSEMBLEIA LEGISLATIVA DO ESTADO DO RN,15
4,002816,2025,EXECUÇÃO DA DECISÃO PROFERIDA NOS AUTOS DO PRO...,DESCONTO EM FOLHA - Implementar,Samya Aby Faraj Linhares Bastos (CPF: 03138324...,6031.7625,ASSEMBLEIA LEGISLATIVA DO ESTADO DO RN,1
5,003034,2025,EXECUÇÃO DA DECISÃO PROFERIDA NOS AUTOS DO PRO...,DESCONTO EM FOLHA - Implementar,EDUARDO FERNANDES PEREIRA (CPF: 05219059408),3296.6670,CÂMARA MUNICIPAL DE SÃO JOSÉ DE CAMPESTRE,1
6,002821,2025,EXECUÇÃO DA DECISÃO PROFERIDA NOS AUTOS DO PRO...,DESCONTO EM FOLHA - Implementar,MANOEL DOS SANTOS BERNARDO (CPF: 02897647426),6565.6931,PREFEITURA MUNICIPAL DE JOÃO CÂMARA,1
7,002820,2025,EXECUÇÃO DA DECISÃO PROFERIDA NOS AUTOS DO PRO...,DESCONTO EM FOLHA - Implementar,SERGIO EDUARDO BEZERRA TEODORO (CPF: 15706141487),11308.9867,"SECRETARIA DE ESTADO DA EDUCAÇÃO, DA CULTURA, ...",1
8,002818,2025,EXECUÇÃO DA DECISÃO PROFERIDA NOS AUTOS DO PRO...,DESCONTO EM FOLHA - Implementar,FRANCISCO BRUNO FERREIRA DA COSTA (CPF: 077605...,831.2759,ASSEMBLEIA LEGISLATIVA DO ESTADO DO RN,1
9,004912,2024,EXECUÇÃO DA DECISÃO PROFERIDA NOS AUTOS DO PRO...,DESCONTO EM FOLHA - Implementar,FÁBIO RODRIGUES DE ARAÚJO (CPF: 80761259449),6672.3091,None,1


In [5]:
descontos_sql_nereu = f''' 
SELECT 
    pro.numero_processo,
    pro.ano_processo,
    pro.assunto,
    pm.Descricao as descricao,
    CONCAT(gp.Nome, ' (CPF: ', gp.Documento, ')') as nome,
    processo.dbo.fn_Exe_RetornaValorAtualizado(ed.IdDebito) as valor_multa,
    'INSTITUTO DE PREVIDÊNCIA SOCIAL DO RIO GRANDE DO NORTE' as nome_orgao,
    ed.CodigoStatusDivida as status_divida
FROM processo.dbo.Processos pro
INNER JOIN processo.dbo.Pro_MarcadorProcesso pmp 
    ON pmp.IdProcesso = pro.IdProcesso
INNER JOIN processo.dbo.Pro_Marcador pm 
    ON pmp.IdMarcador = pm.IdMarcador
LEFT JOIN processo.dbo.Exe_Debito ed 
    ON ed.IdProcessoExecucao = pro.IdProcesso
LEFT JOIN processo.dbo.Exe_DebitoPessoa edp 
    ON edp.IDDebito = ed.IdDebito
LEFT JOIN processo.dbo.GenPessoa gp 
    ON gp.IdPessoa = edp.IDPessoa
WHERE 
    pro.setor_atual = 'CCD'
    AND CONCAT(pro.numero_processo, '/', pro.ano_processo) IN ('001361/2022',
'003654/2022',
'002748/2022',
'000133/2022',
'002739/2022',
'003061/2022',
'000152/2022',
'000129/2022',
'003054/2022',
'003010/2022',
'003010/2022',
'000112/2022',
'000116/2022',
'000115/2022',
'000111/2022',
'000125/2022',
'000128/2022',
'000157/2022',
'000123/2022',
'001425/2022',
'002716/2022',
'001365/2022',
'001423/2022',
'001362/2022',
'001366/2022',
'000126/2022',
'000131/2022',
'002682/2022',
'003006/2022',
'000119/2022',
'002062/2024',
'002038/2024',
'000470/2024',
'000142/2023',
'000101/2022',
'003078/2022',
'003023/2022',
'003668/2022',
'003690/2022',
'003709/2022',
'003687/2022',
'000421/2024',
'003659/2022',
'003667/2022',
'003663/2022',
'003064/2022',
'000088/2023',
'003686/2022',
'000157/2023',
'003701/2022',
'003263/2023',
'003700/2022',
'000480/2024',
'012970/2017'
) 
    AND ed.CodigoTipoDebito NOT IN (1, 3)
'''
#processos_descontos = pd.read_sql(descontos_sql_nereu, conn)

In [8]:
processos_descontos = processos_descontos[~processos_descontos.nome_orgao.isna()]

In [9]:
def criar_valor_extenso(x):
        if x and x > 1:
            f = lambda c: num2words(int(c), lang='pt_BR')
            return ' reais e '.join([f(y) for y in str('{:.2f}'.format(x)).split('.')]) + ' centavos'
        else:
            return ''

def converter_valor(x):
    if x:
        return locale.currency(x, grouping=True) + ' (' + criar_valor_extenso(x) + ')'
    else:
        return 'R$ 0,00 (zero)'

In [10]:
processos_descontos.fillna(0, inplace=True)

C:\Users\05911205424\AppData\Local\Temp\ipykernel_9704\3848932403.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  processos_descontos.fillna(0, inplace=True)


In [11]:
processos_descontos = processos_descontos.groupby(['numero_processo', 'ano_processo', 'assunto', 'descricao', 'nome'
    ,'nome_orgao', 'status_divida']).valor_multa.sum().reset_index() 

In [12]:
processos_descontos['processo'] = processos_descontos['numero_processo'] + '/' + processos_descontos['ano_processo'].astype(str)
processos_descontos['valor'] = processos_descontos['valor_multa'].apply(converter_valor)  

In [13]:
processos_descontos

,numero_processo,ano_processo,assunto,descricao,nome,nome_orgao,status_divida,valor_multa,processo,valor
0,002138,2019,EXECUÇÃO DA DECISÃO PROFERIDA NOS AUTOS DO PRO...,DESCONTO EM FOLHA - Implementar,LUDMILA CARLOS AMORIM DE ARAÚJO ROSADO (CPF: 0...,PREFEITURA MUNICIPAL DE RAFAEL GODEIRO,1,22638.7932,002138/2019,"R$ 22.638,79 (vinte e dois mil, seiscentos e t..."
1,002816,2025,EXECUÇÃO DA DECISÃO PROFERIDA NOS AUTOS DO PRO...,DESCONTO EM FOLHA - Implementar,Samya Aby Faraj Linhares Bastos (CPF: 03138324...,ASSEMBLEIA LEGISLATIVA DO ESTADO DO RN,1,6031.7625,002816/2025,"R$ 6.031,76 (seis mil e trinta e um reais e se..."
2,002818,2025,EXECUÇÃO DA DECISÃO PROFERIDA NOS AUTOS DO PRO...,DESCONTO EM FOLHA - Implementar,FRANCISCO BRUNO FERREIRA DA COSTA (CPF: 077605...,ASSEMBLEIA LEGISLATIVA DO ESTADO DO RN,1,831.2759,002818/2025,"R$ 831,28 (oitocentos e trinta e um reais e vi..."
3,002818,2025,EXECUÇÃO DA DECISÃO PROFERIDA NOS AUTOS DO PRO...,DESCONTO EM FOLHA - Implementar,FRANCISCO BRUNO FERREIRA DA COSTA (CPF: 077605...,ASSEMBLEIA LEGISLATIVA DO ESTADO DO RN,15,828.5144,002818/2025,"R$ 828,51 (oitocentos e vinte e oito reais e c..."
4,002819,2025,EXECUÇÃO DA DECISÃO PROFERIDA NOS AUTOS DO PRO...,DESCONTO EM FOLHA - Implementar,CARLOS ALBERTO JÁCOME DE AQUINO (CPF: 46554408...,SECRETARIA DE ESTADO DA TRIBUTAÇÃO,1,16624.8640,002819/2025,"R$ 16.624,86 (dezesseis mil, seiscentos e vint..."
5,002820,2025,EXECUÇÃO DA DECISÃO PROFERIDA NOS AUTOS DO PRO...,DESCONTO EM FOLHA - Implementar,SERGIO EDUARDO BEZERRA TEODORO (CPF: 15706141487),"SECRETARIA DE ESTADO DA EDUCAÇÃO, DA CULTURA, ...",1,11308.9867,002820/2025,"R$ 11.308,99 (onze mil, trezentos e oito reais..."
6,002821,2025,EXECUÇÃO DA DECISÃO PROFERIDA NOS AUTOS DO PRO...,DESCONTO EM FOLHA - Implementar,MANOEL DOS SANTOS BERNARDO (CPF: 02897647426),PREFEITURA MUNICIPAL DE JOÃO CÂMARA,1,6565.6931,002821/2025,"R$ 6.565,69 (seis mil, quinhentos e sessenta e..."
7,003003,2021,EXECUÇÃO DA DECISÃO PROFERIDA NOS AUTOS DO PRO...,DESCONTO EM FOLHA - Implementar,CARLOS VERIANO DE LIMA (CPF: 03245923457),PREFEITURA MUNICIPAL DE PARAZINHO,1,14983.4596,003003/2021,"R$ 14.983,46 (catorze mil, novecentos e oitent..."
8,003034,2025,EXECUÇÃO DA DECISÃO PROFERIDA NOS AUTOS DO PRO...,DESCONTO EM FOLHA - Implementar,EDUARDO FERNANDES PEREIRA (CPF: 05219059408),CÂMARA MUNICIPAL DE SÃO JOSÉ DE CAMPESTRE,1,3296.6670,003034/2025,"R$ 3.296,67 (três mil, duzentos e noventa e se..."
9,003037,2025,EXECUÇÃO DA DECISÃO PROFERIDA NOS AUTOS DO PRO...,DESCONTO EM FOLHA - Implementar,MARCOS ANTÔNIO CABRAL (CPF: 32403143472),ASSEMBLEIA LEGISLATIVA DO ESTADO DO RN,1,1195.2656,003037/2025,"R$ 1.195,27 (mil, cento e noventa e cinco reai..."


In [14]:
descontos_aberto = processos_descontos[processos_descontos.status_divida == 1]

In [15]:
descontos_aberto

,numero_processo,ano_processo,assunto,descricao,nome,nome_orgao,status_divida,valor_multa,processo,valor
0,002138,2019,EXECUÇÃO DA DECISÃO PROFERIDA NOS AUTOS DO PRO...,DESCONTO EM FOLHA - Implementar,LUDMILA CARLOS AMORIM DE ARAÚJO ROSADO (CPF: 0...,PREFEITURA MUNICIPAL DE RAFAEL GODEIRO,1,22638.7932,002138/2019,"R$ 22.638,79 (vinte e dois mil, seiscentos e t..."
1,002816,2025,EXECUÇÃO DA DECISÃO PROFERIDA NOS AUTOS DO PRO...,DESCONTO EM FOLHA - Implementar,Samya Aby Faraj Linhares Bastos (CPF: 03138324...,ASSEMBLEIA LEGISLATIVA DO ESTADO DO RN,1,6031.7625,002816/2025,"R$ 6.031,76 (seis mil e trinta e um reais e se..."
2,002818,2025,EXECUÇÃO DA DECISÃO PROFERIDA NOS AUTOS DO PRO...,DESCONTO EM FOLHA - Implementar,FRANCISCO BRUNO FERREIRA DA COSTA (CPF: 077605...,ASSEMBLEIA LEGISLATIVA DO ESTADO DO RN,1,831.2759,002818/2025,"R$ 831,28 (oitocentos e trinta e um reais e vi..."
4,002819,2025,EXECUÇÃO DA DECISÃO PROFERIDA NOS AUTOS DO PRO...,DESCONTO EM FOLHA - Implementar,CARLOS ALBERTO JÁCOME DE AQUINO (CPF: 46554408...,SECRETARIA DE ESTADO DA TRIBUTAÇÃO,1,16624.8640,002819/2025,"R$ 16.624,86 (dezesseis mil, seiscentos e vint..."
5,002820,2025,EXECUÇÃO DA DECISÃO PROFERIDA NOS AUTOS DO PRO...,DESCONTO EM FOLHA - Implementar,SERGIO EDUARDO BEZERRA TEODORO (CPF: 15706141487),"SECRETARIA DE ESTADO DA EDUCAÇÃO, DA CULTURA, ...",1,11308.9867,002820/2025,"R$ 11.308,99 (onze mil, trezentos e oito reais..."
6,002821,2025,EXECUÇÃO DA DECISÃO PROFERIDA NOS AUTOS DO PRO...,DESCONTO EM FOLHA - Implementar,MANOEL DOS SANTOS BERNARDO (CPF: 02897647426),PREFEITURA MUNICIPAL DE JOÃO CÂMARA,1,6565.6931,002821/2025,"R$ 6.565,69 (seis mil, quinhentos e sessenta e..."
7,003003,2021,EXECUÇÃO DA DECISÃO PROFERIDA NOS AUTOS DO PRO...,DESCONTO EM FOLHA - Implementar,CARLOS VERIANO DE LIMA (CPF: 03245923457),PREFEITURA MUNICIPAL DE PARAZINHO,1,14983.4596,003003/2021,"R$ 14.983,46 (catorze mil, novecentos e oitent..."
8,003034,2025,EXECUÇÃO DA DECISÃO PROFERIDA NOS AUTOS DO PRO...,DESCONTO EM FOLHA - Implementar,EDUARDO FERNANDES PEREIRA (CPF: 05219059408),CÂMARA MUNICIPAL DE SÃO JOSÉ DE CAMPESTRE,1,3296.6670,003034/2025,"R$ 3.296,67 (três mil, duzentos e noventa e se..."
9,003037,2025,EXECUÇÃO DA DECISÃO PROFERIDA NOS AUTOS DO PRO...,DESCONTO EM FOLHA - Implementar,MARCOS ANTÔNIO CABRAL (CPF: 32403143472),ASSEMBLEIA LEGISLATIVA DO ESTADO DO RN,1,1195.2656,003037/2025,"R$ 1.195,27 (mil, cento e noventa e cinco reai..."
10,004912,2024,EXECUÇÃO DA DECISÃO PROFERIDA NOS AUTOS DO PRO...,DESCONTO EM FOLHA - Implementar,AFRÂNIO REIS CAVALCANTE (CPF: 09477225487),INSTITUTO DE PREVIDÊNCIA DOS SERVIDORES DO RN,1,32470.0141,004912/2024,"R$ 32.470,01 (trinta e dois mil, quatrocentos ..."


In [16]:
for desconto in descontos_aberto.itertuples():
    doc = DocxTemplate("templates/desconto_folha.docx")
    context = {
        'processo': desconto.processo,
        'assunto': desconto.assunto,
        'nome': desconto.nome,
        'orgao': desconto.nome_orgao,
        'valor': desconto.valor
    }
    nome = desconto.nome.split()[0].lower()
    processo_underline = desconto.processo.replace('/', '_')

    doc.render(context)
    d = f"saidas/desconto_folha/{processo_underline}_{nome}.docx"
    doc.save(d)
    generate_pdf_office(d, "saidas/desconto_folha/")
    print(f"Gerado arquivo para o processo {desconto.processo}")

Gerado arquivo para o processo 002138/2019
Gerado arquivo para o processo 002816/2025
Gerado arquivo para o processo 002818/2025
Gerado arquivo para o processo 002819/2025
Gerado arquivo para o processo 002820/2025
Gerado arquivo para o processo 002821/2025
Gerado arquivo para o processo 003003/2021
Gerado arquivo para o processo 003034/2025
Gerado arquivo para o processo 003037/2025
Gerado arquivo para o processo 004912/2024
